In [2]:
import matplotlib.pyplot as plt
import urllib.request
import os
import gzip
import struct
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score

from DBNAC import DBNClassifier, SimpleDBNClassifier
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-images-idx3-ubyte.gz',
#                            filename=os.path.join(data_folder, 'train-images.gz'))
# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/train-labels-idx1-ubyte.gz',
#                            filename=os.path.join(data_folder, 'train-labels.gz'))
# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-images-idx3-ubyte.gz',
#                            filename=os.path.join(data_folder, 'test-images.gz'))
# urllib.request.urlretrieve('https://azureopendatastorage.blob.core.windows.net/mnist/t10k-labels-idx1-ubyte.gz',
#                            filename=os.path.join(data_folder, 'test-labels.gz'))
# load compressed MNIST gz files and return numpy arrays
def load_data(filename, label=False):
    with gzip.open(filename) as gz:
        struct.unpack('I', gz.read(4))
        n_items = struct.unpack('>I', gz.read(4))
        if not label:
            n_rows = struct.unpack('>I', gz.read(4))[0]
            n_cols = struct.unpack('>I', gz.read(4))[0]
            res = np.frombuffer(gz.read(n_items[0] * n_rows * n_cols), dtype=np.uint8)
            res = res.reshape(n_items[0], n_rows * n_cols)
        else:
            res = np.frombuffer(gz.read(n_items[0]), dtype=np.uint8)
            res = res.reshape(n_items[0], 1)
    return res

In [5]:
# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(os.path.join(
    data_folder, 'train-images.gz'), False)
X_test = load_data(os.path.join(data_folder, 'test-images.gz'), False)
y_train = load_data(os.path.join(
    data_folder, 'train-labels.gz'), True).reshape(-1)
y_test = load_data(os.path.join(
    data_folder, 'test-labels.gz'), True).reshape(-1)
X_train = (X_train > 127)*1
X_test = (X_test > 127)*1

In [11]:
def acc(pred, y):
    return np.mean((pred==y)*1.0)

In [24]:
list_i = [1000,3000,7000,10000,60000]
for i in list_i:
    clf = SimpleDBNClassifier(n_hiddens=[200, 200, 10], lr_pre_train=1e-5, lr_fine_tune=0.01, 
                    epochs_pre_train=20, epochs_fine_tune=20, batch_size=50, k=3, 
                    random_state=42, use_gpu=True, verbose=False)
    # scores = cross_val_score(clf, X_train, y_train)
    # print(np.mean(scores))
    # print(scores)
    x = X_train[:i]
    y = y_train[:i]
    clf.fit(x,y)
    print('i=',i)
    print('Accuracy train', acc(clf.predict(x), y))
    print('Accuracy test', acc(clf.predict(X_test), y_test))

i= 1000
Accuracy train 0.999
Accuracy test 0.8697
i= 3000
Accuracy train 0.998
Accuracy test 0.9208
i= 7000
Accuracy train 0.9884285714285714
Accuracy test 0.9233
i= 10000
Accuracy train 0.9885
Accuracy test 0.9364
